In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/19 10:05
# @Author  : Wang Yujia
# @File    : data_extract_for_targets.ipynb
# @Description : 为了target datas提取信息

# 1. Preparations
## 1.1 全局设置
1. input的data包括那些duration过长的结果
2. output一共`data_key.shape[0]`+1个文件

In [7]:
data_path = "../data/datawithnp_2_selected.csv"

# output path
target_output_head= "../data/targets/target_data_NP_"
target_output_tail= ".csv"
target_data_key_path = "../data/targets/target_datakey.csv"

features = ['id', 'bidincrement', 'bidfee','retail']
import pandas as pd
import numpy as np
from visdom import Visdom
import csv
from tqdm import tqdm

## 1.2 data读取与保存data_key

In [8]:
data = pd.read_csv(data_path, encoding="utf-8")
data.head()

,auction_id,product_id,item,desc,retail,price,finalprice,bidincrement,bidfee,winner,...,endtime_str,flg_click_only,flg_beginnerauction,flg_fixedprice,flg_endprice,id,N,cnt_uniq,cnt_n_2,P
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,13.35,0.15,0.75,Racer11,...,19:52 PDT 09-16-2008,0,0,0,0,0,89,69,1,0.014493
1,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,74.70,0.15,0.75,Cemo23,...,11:17 PDT 08-28-2008,0,0,0,0,1,498,60,1,0.016667
2,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,83.10,0.15,0.75,Jacobsonnich,...,22:52 PDT 11-07-2008,0,1,0,0,1,554,60,1,0.016667
3,88638,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,19.65,0.15,0.75,Mokkis,...,22:02 PDT 08-23-2008,0,0,0,0,0,131,69,1,0.014493
4,88639,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,47.10,0.15,0.75,Superloeffel,...,14:23 PDT 08-24-2008,0,0,0,0,0,314,69,1,0.014493


In [10]:
data_key = data[features].copy()
data_key.drop_duplicates(inplace=True,ignore_index=True)
# make sure int
data_key['id'] = data_key['id'].astype(int)
data_key.to_csv(target_data_key_path,header=True, encoding="utf-8",index=False)
print(f"target data中，一共包括 *{data_key.shape[0]}* 个data keys")
data_key.head()

target data中，一共包括 *1196* 个data keys


,id,bidincrement,bidfee,retail
0,0,0.15,0.75,499.99
1,1,0.15,0.75,169.99
2,2,0.15,0.75,299.99
3,3,0.15,0.75,89.99
4,5,0.15,0.75,59.99


## 1.3 functions about 'key'

In [11]:
# get key from i in 'data_key'
def get_key_from_index(i,flag="NotStr"):
    if(flag == "str"):
        key_i = list(data_key.iloc[i,:])
        key_i_str = (str(int(key_i[0])),str(key_i[1]),str(key_i[2]),str(key_i[3]))   # 'product_id'是int类型
        return key_i_str
    else:
        key_i = data_key.iloc[i,:]
        return key_i

def select_data_fromkey(key_i_str):
    return data[(data['id'] == key_i_str[0]) & (data['bidincrement'] == key_i_str[1]) & (data['bidfee'] == key_i_str[2]) & (data['retail'] == key_i_str[3])].copy()

# 2. 整理N
## 2.1 N
1. 整理成一个[key]对应一个“N” vector的格式


In [13]:
plot_flag = 0
if(plot_flag):
    env_str = 'N_targets'
    viz = Visdom(env=env_str)

col_names = ['id','bidincrement','bidfee','retail','N','P','cnt_n_2']

print("Done")

Done


In [14]:
# data_key.shape[0]
for i in tqdm(range(0,data_key.shape[0])):

    # get i_th data_key
    key_i = get_key_from_index(i)
    data_i = select_data_fromkey(key_i)
    data_i.reset_index(drop=True,inplace=True)
    data_i.sort_values(by='N',ignore_index=True,inplace=True)
    data_i_df = data_i[col_names]

    # get the filepath and output
    target_output_path = target_output_head+str(i)+target_output_tail
    data_i_df.to_csv(target_output_path,header=True,index=False,encoding="utf-8")

print("Done")

100%|██████████| 1196/1196 [00:02<00:00, 477.46it/s]

Done


# 3 save
1. 保存结果以及viz环境

In [ ]:
# 保存整个环境
if(plot_flag):
    viz.save(envs=[env_str])